In [15]:
import os
import pandas as pd
from google.colab import files

file_name = 'data_center_prob_estat.csv'

if not os.path.exists(file_name):
    print(f"Por favor, faça o upload do arquivo: {file_name}")
    uploaded = files.upload()
    for fn in uploaded.keys():
        print(f'Arquivo "{fn}" carregado com sucesso.')
else:
    print(f"Arquivo {file_name} já existe no ambiente.")

# Carregar o arquivo CSV original
df = pd.read_csv('data_center_prob_estat.csv')

# 1. Verificar as dimensões do conjunto de dados
print('Dimensões do conjunto de dados:')
print(f'Linhas: {df.shape[0]}, Colunas: {df.shape[1]}')

# 2. Inspecionar os tipos de dados de cada coluna
print('\nTipos de dados de cada coluna:')
print(df.dtypes)

# 3. Identificar a presença de valores ausentes por coluna
print('\nValores ausentes por coluna:')
print(df.isnull().sum())

Arquivo data_center_prob_estat.csv já existe no ambiente.
Dimensões do conjunto de dados:
Linhas: 720, Colunas: 12

Tipos de dados de cada coluna:
timestamp                    object
hora_dia                    float64
dia_semana                  float64
reqs                        float64
trafego_MBps                float64
usuarios_ativos             float64
temp_ambiente_C             float64
uso_cpu_%                   float64
uso_ram_%                   float64
demanda_energia_kW          float64
pressao_resfriamento_bar    float64
consumo_agua_m3             float64
dtype: object

Valores ausentes por coluna:
timestamp                   12
hora_dia                     4
dia_semana                   7
reqs                         8
trafego_MBps                 6
usuarios_ativos             11
temp_ambiente_C             11
uso_cpu_%                   10
uso_ram_%                   10
demanda_energia_kW           7
pressao_resfriamento_bar     6
consumo_agua_m3             11
dtype

In [16]:
# Identificar o número de linhas duplicadas
num_duplicates = df.duplicated().sum()
print(f'Número de linhas duplicadas encontradas: {num_duplicates}')

# Remover linhas duplicadas
df_cleaned = df.drop_duplicates()
print(f'Número de linhas após a remoção de duplicados: {df_cleaned.shape[0]}')

# Atualizar df para a próxima etapa
df = df_cleaned.copy()

Número de linhas duplicadas encontradas: 0
Número de linhas após a remoção de duplicados: 720


In [17]:
# Converter a coluna 'timestamp' para datetime
# Usar errors='coerce' para transformar valores inválidos em NaT (Not a Time)
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')

# Verificar os tipos de dados após a conversão
print("\nTipos de dados após correção:")
print(df.dtypes)


Tipos de dados após correção:
timestamp                   datetime64[ns]
hora_dia                           float64
dia_semana                         float64
reqs                               float64
trafego_MBps                       float64
usuarios_ativos                    float64
temp_ambiente_C                    float64
uso_cpu_%                          float64
uso_ram_%                          float64
demanda_energia_kW                 float64
pressao_resfriamento_bar           float64
consumo_agua_m3                    float64
dtype: object


In [18]:
# Identificar colunas numéricas (excluindo 'timestamp', 'hora_dia', 'dia_semana' se forem categóricas ou de tempo)
numeric_cols = df.select_dtypes(include=["float64", "int64"]).columns.tolist()

# Excluir colunas que não devem ter outliers tratados (e.g., identificadores, tempo)
# Assumindo que 'hora_dia' e 'dia_semana' são categóricas ou ordinais e não devem ter outliers tratados por IQR
if 'hora_dia' in numeric_cols:
    numeric_cols.remove('hora_dia')
if 'dia_semana' in numeric_cols:
    numeric_cols.remove('dia_semana')

print("Colunas numéricas para tratamento de outliers:", numeric_cols)

# Função para tratar outliers usando o método IQR
def treat_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Substituir outliers pela mediana dos valores não-outliers
    median_non_outliers = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)][column].median()
    df.loc[(df[column] < lower_bound) | (df[column] > upper_bound), column] = median_non_outliers
    return df

# Aplicar o tratamento de outliers para cada coluna numérica
for col in numeric_cols:
    df = treat_outliers_iqr(df, col)

Colunas numéricas para tratamento de outliers: ['reqs', 'trafego_MBps', 'usuarios_ativos', 'temp_ambiente_C', 'uso_cpu_%', 'uso_ram_%', 'demanda_energia_kW', 'pressao_resfriamento_bar', 'consumo_agua_m3']


In [19]:
# Remover linhas onde 'timestamp' é NaT (valores inválidos após conversão)
df.dropna(subset=['timestamp'], inplace=True)

# Definir 'timestamp' como índice e ordenar
df = df.set_index("timestamp").sort_index()

# Aplicar interpolação baseada no tempo
# Para colunas numéricas, use interpolate com method='time'
for col in df.select_dtypes(include=["float64", "int64"]).columns:
    df[col] = df[col].interpolate(method='time')

# Preencher quaisquer valores NaN restantes (por exemplo, no início/fim da série) com ffill/bfill
df = df.fillna(method='ffill').fillna(method='bfill')

# Resetar o índice para que 'timestamp' volte a ser uma coluna regular
df = df.reset_index()

print("\nValores ausentes após a interpolação:")
print(df.isnull().sum())


Valores ausentes após a interpolação:
timestamp                   0
hora_dia                    0
dia_semana                  0
reqs                        0
trafego_MBps                0
usuarios_ativos             0
temp_ambiente_C             0
uso_cpu_%                   0
uso_ram_%                   0
demanda_energia_kW          0
pressao_resfriamento_bar    0
consumo_agua_m3             0
dtype: int64


/tmp/ipython-input-829579234.py:13: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='ffill').fillna(method='bfill')


In [20]:
# Salvar o DataFrame final em um novo arquivo CSV
df.to_csv("data_cleaned_final.csv", index=False)
print("DataFrame final salvo como: data_cleaned_final.csv")

DataFrame final salvo como: data_cleaned_final.csv
